In [5]:
import pandas as pd
import numpy as np
import re

# ===== 파일 경로 =====
WILDFIRE_PATH  = "/Users/igangsan/Desktop/ML/wildfire_2019_2023_combine.csv"
FOREST_PATH    = "/Users/igangsan/Desktop/ML/실험/forest_area_with_full_region.xlsx"
OUT_CSV        = "/Users/igangsan/Desktop/forest_added.csv"

# ===== 광역단위 축약(최신 명칭 포함) =====
abbr_map = {
    "서울특별시":"서울", "부산광역시":"부산", "대구광역시":"대구", "인천광역시":"인천",
    "광주광역시":"광주", "대전광역시":"대전", "울산광역시":"울산",
    "세종특별자치시":"세종", "세종특별자치도":"세종",
    "경기도":"경기", "강원도":"강원", "강원특별자치도":"강원",
    "충청북도":"충북", "충청남도":"충남",
    "전라북도":"전북", "전북특별자치도":"전북",
    "전라남도":"전남",
    "경상북도":"경북", "경상남도":"경남",
    "제주특별자치도":"제주"
}
DIRECTION_FIX = {"북":"북구","남":"남구","동":"동구","서":"서구","중":"중구"}

def clean_text(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    s = re.sub(r"[\(\（].*?[\)\）]", "", s)
    s = re.sub(r"[,/·\-]", " ", s)
    s = s.replace("\u00a0", " ").replace("\u200b", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def norm_province(s: str) -> str:
    return abbr_map.get(clean_text(s), clean_text(s))

def strip_suffix_district(tok: str) -> str:
    return re.sub(r"(자치구|구|군|시)$", "", tok.strip())

def fix_direction_only(tok: str) -> str:
    return DIRECTION_FIX.get(tok, tok)

def norm_district(dist: str) -> str:
    dist = clean_text(dist)
    toks = [t for t in dist.split(" ") if t]
    toks = [fix_direction_only(t) for t in toks]
    toks = [strip_suffix_district(t) for t in toks]
    toks = [t for t in toks if t]
    return " ".join(toks)

def norm_pair(prov: str, dist: str) -> str:
    p = norm_province(prov)
    d = norm_district(dist)
    if p == "세종" and (d == "" or d == "세종"):
        return "세종"
    return f"{p} {d}".strip()

def norm_single(full_region: str) -> str:
    toks = clean_text(full_region).split()
    if not toks: return ""
    p, rest = toks[0], " ".join(toks[1:])
    return norm_pair(p, rest)

def city_token(dist: str) -> str:
    dist = norm_district(dist)
    return dist.split()[0] if dist else ""

def to_number(x):
    if pd.isna(x): return x
    s = str(x).replace(",", "").replace("\u00a0","").strip()
    return pd.to_numeric(s, errors="ignore")

# ---------- 시간 파싱 유틸 ----------
def _to_str_keep_digits(x):
    if pd.isna(x):
        return ""
    # 숫자(특히 float 과학표기 2.01903E+13) → 정수 문자열로 안정 변환
    if isinstance(x, (int, np.integer)):
        return f"{int(x)}"
    if isinstance(x, (float, np.floating)):
        # 소수점 없는 큰 정수로 가정하고 반올림
        return f"{int(round(float(x)))}"
    return str(x).strip()

def parse_any_datetime(x):
    """다양한 형식을 견고하게 파싱 → pandas.Timestamp 또는 NaT"""
    s = _to_str_keep_digits(x)
    if s == "":
        return pd.NaT

    # 1) 순수 숫자(YYYYMMDDHHMMSS/ YYYYMMDDHHMM / YYYYMMDD)
    if s.isdigit():
        if len(s) >= 14:  # 최소 14자리 이상 오면 앞 14자리 사용
            s14 = s[:14]
            try:
                return pd.to_datetime(s14, format="%Y%m%d%H%M%S", errors="coerce")
            except Exception:
                pass
        elif len(s) == 12:  # YYYYMMDDHHMM
            try:
                return pd.to_datetime(s, format="%Y%m%d%H%M", errors="coerce")
            except Exception:
                pass
        elif len(s) == 8:   # YYYYMMDD
            try:
                return pd.to_datetime(s, format="%Y%m%d", errors="coerce")
            except Exception:
                pass

    # 2) ISO / 공백구분 / 점/슬래시/하이픈 / AM/PM 등 일반적 문자열
    #    예: 2019-03-07 12:14:00, 2019.1.2 4:10:00 PM, 2019/03/07T12:14:00
    ts = pd.to_datetime(s, errors="coerce", infer_datetime_format=True)
    if pd.isna(ts):
        # 흔한 변형들 마지막 시도
        s2 = s.replace(".", "-").replace("/", "-").replace("T", " ")
        ts = pd.to_datetime(s2, errors="coerce", infer_datetime_format=True)
    return ts

def pick_col(df, base_name: str):
    """대소문자 무시 + 접미사(.1, _x, _y 등) 허용해서 실제 컬럼명 찾아오기"""
    base = base_name.lower()
    # 완전 일치 우선
    for c in df.columns:
        if c.lower() == base:
            return c
    # 접두 일치(예: rcpt_dt.1, rcpt_dt_x)
    cand = [c for c in df.columns if c.lower().startswith(base)]
    return cand[0] if cand else None

def ensure_canonical_time_cols(df, canonical_names):
    """
    각 canonical 이름에 대해 실제 존재하는 컬럼 찾아서
    df[canonical] = df[src] 형태로 복제(없으면 건너뜀).
    이후 canonical 이름만 사용해 파싱/계산.
    """
    mapping = {}
    for name in canonical_names:
        src = pick_col(df, name)
        if src is not None:
            if name not in df.columns:
                df[name] = df[src]
            mapping[name] = name  # 이제 canonical 보장
    return df, mapping

def main():
    # 1) 로드
    wild = pd.read_csv(WILDFIRE_PATH)
    forest = pd.read_excel(FOREST_PATH)

    # 2) 키 생성
    wild["__KEY_MAIN__"] = wild.apply(lambda r: norm_pair(r["CTPV_NM"], r["SGG_NM"]), axis=1)
    wild["__KEY_CITY__"] = wild.apply(lambda r: f"{norm_province(r['CTPV_NM'])} {city_token(r['SGG_NM'])}".strip(), axis=1)
    wild["__KEY_SIDO__"] = wild["CTPV_NM"].map(norm_province)

    forest["__KEY_MAIN__"] = forest["행정구역_full"].map(norm_single)
    def forest_city_key(s: str) -> str:
        toks = clean_text(s).split()
        if not toks: return ""
        p = norm_province(toks[0])
        rest = " ".join(toks[1:]) if len(toks) > 1 else ""
        c = city_token(rest)
        if p == "세종" and (c == "" or c == "세종"):
            return "세종"
        return f"{p} {c}".strip()
    forest["__KEY_CITY__"] = forest["행정구역_full"].map(forest_city_key)
    forest["__KEY_SIDO__"] = forest["행정구역_full"].map(lambda s: norm_province(clean_text(s).split()[0]) if clean_text(s) else "")

    # 3) forest 집계
    forest_cols = [c for c in forest.columns if c not in {"행정구역_full","__KEY_MAIN__","__KEY_CITY__","__KEY_SIDO__"}]
    for c in forest_cols:
        forest[c] = forest[c].map(to_number)
    agg = {c: ("mean" if pd.api.types.is_numeric_dtype(forest[c]) else "first") for c in forest_cols}

    forest_main = forest.groupby("__KEY_MAIN__", as_index=False).agg(agg)
    forest_city = forest.groupby("__KEY_CITY__", as_index=False).agg(agg)
    forest_sido = forest.groupby("__KEY_SIDO__", as_index=False).agg(agg)

    # 4) 다단계 병합
    m = wild.copy()
    m = m.merge(forest_main, on="__KEY_MAIN__", how="left")
    mask = m[forest_cols].notna().any(axis=1)

    need = ~mask
    fill2 = m.loc[need, ["__KEY_CITY__"]].join(
        forest_city.set_index("__KEY_CITY__"), on="__KEY_CITY__", how="left"
    )[forest_cols]
    m.loc[need, forest_cols] = fill2.values
    mask2 = m[forest_cols].notna().any(axis=1)

    need3 = ~mask2
    fill3 = m.loc[need3, ["__KEY_SIDO__"]].join(
        forest_sido.set_index("__KEY_SIDO__"), on="__KEY_SIDO__", how="left"
    )[forest_cols]
    m.loc[need3, forest_cols] = fill3.values

    # 5) 컬럼명 전부 소문자
    m.columns = [c.lower() for c in m.columns]

    # 6) 시간 컬럼 정규화(대소문자/접미사 허용 → canonical 복제)
    canonical = ["rcpt_dt", "dspt_dt", "grnds_arvl_dt", "bgnn_potfr_dt", "prfect_potfr_dt", "cbk_dt"]
    m, _ = ensure_canonical_time_cols(m, canonical)

    # 7) 시간 파싱(모든 포맷 허용)
    for c in canonical:
        if c in m.columns:
            m[c] = m[c].apply(parse_any_datetime)

    # 8) 파생(초 단위)
    def diff_seconds(later, earlier):
        if pd.isna(later) or pd.isna(earlier):
            return np.nan
        return (later - earlier).total_seconds()

    if all(c in m.columns for c in ["cbk_dt","rcpt_dt"]):
        m["total_time"] = [diff_seconds(l, e) for l, e in zip(m["cbk_dt"], m["rcpt_dt"])]

    if all(c in m.columns for c in ["prfect_potfr_dt","dspt_dt"]):
        m["fire_supesn_hr"] = [diff_seconds(l, e) for l, e in zip(m["prfect_potfr_dt"], m["dspt_dt"])]

    if all(c in m.columns for c in ["grnds_arvl_dt","dspt_dt"]):
        m["dspt_req_hr"] = [diff_seconds(l, e) for l, e in zip(m["grnds_arvl_dt"], m["dspt_dt"])]

    if all(c in m.columns for c in ["grnds_arvl_dt","rcpt_dt"]):
        m["arrival_time_diff"] = [diff_seconds(l, e) for l, e in zip(m["grnds_arvl_dt"], m["rcpt_dt"])]

    if all(c in m.columns for c in ["prfect_potfr_dt","bgnn_potfr_dt"]):
        m["relax_diff"] = [diff_seconds(l, e) for l, e in zip(m["prfect_potfr_dt"], m["bgnn_potfr_dt"])]

    if all(c in m.columns for c in ["dspt_dt","rcpt_dt"]):
        m["dispatch_time_diff"] = [diff_seconds(l, e) for l, e in zip(m["dspt_dt"], m["rcpt_dt"])]

    if "dspt_req_hr" in m.columns:
        m["golden_time_under_50min"] = ((m["dspt_req_hr"] < 50*60).fillna(False)).astype(int)

    def is_night(ts):
        if pd.isna(ts): return 0
        h = ts.hour
        return 1 if (h >= 20 or h < 7) else 0

    if "rcpt_dt" in m.columns:
        m["is_night"] = m["rcpt_dt"].apply(is_night).astype(int)
        m["month_rcpt"] = m["rcpt_dt"].dt.month

    # 9) 내부 키 제거 후 저장
    m = m.drop(columns=["__key_main__","__key_city__","__key_sido__"], errors="ignore")
    m.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print("저장 완료:", OUT_CSV)

if __name__ == "__main__":
    main()


/var/folders/h1/2kvn4fr11vn6f3469636zhp40000gn/T/ipykernel_5781/3934359344.py:69: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.to_numeric(s, errors="ignore")


저장 완료: /Users/igangsan/Desktop/forest_added.csv
